In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import time
import pathlib
python_path = pathlib.Path('.').absolute().parent/'python'
os.sys.path.insert(1, str(python_path))

In [ ]:
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import trange
import wandb

In [ ]:
from vocam.diff_pin_costs import DiffFrameTranslationCost, DiffFrameVelocityCost
from vocam.data_gen import generate_obstacle

In [ ]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")
j_id_arr = [f_id]
for i in range(3, model.nq + 1):
    j_id_arr.append(model.getFrameId("A" + str(i)))

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

In [ ]:
run = wandb.init(project="vocam", group="data_obstacle", entity="hjzhu")

In [ ]:
# robot parameters
wandb.config.nq = model.nq
wandb.config.nv = model.nv
wandb.config.n_col = 5
wandb.config.n_vars = 3 * model.nq * wandb.config.n_col + 2 * model.nq
wandb.config.u_max = [2.5, 2.5, 2.5, 1.5, 1.5, 1.5, 1.0]
wandb.config.dt = 0.05
wandb.config.q_init = np.array([2.1789238e-02,  3.3214998e-01, -1.4518893e-04, -8.7141126e-01,
                                6.0329604e-01, -1.3965217e-03,  1.4794523e-04])
wandb.config.x_init = np.concatenate([wandb.config.q_init, pin.utils.zero(model.nv)])
wandb.config.q_noise = 0.6
wandb.config.dq_noise = 1.4
wandb.config.f_id = f_id
wandb.config.j_id_arr = j_id_arr

# ioc optimization parameters
wandb.config.isvec = True
wandb.config.lr_qp = 1e-1
wandb.config.max_it = 100
wandb.config.task_horizon = 20
wandb.config.loss_threshold = 0.05
wandb.config.convergence_threshold = 5e-4
wandb.config.distance_threshold = 0.10

# obstacle parameters
wandb.config.n_obs = 1
wandb.config.obs_rad = [0.5, 0.35, 0.4] 
wandb.config.n_restart = 5

In [ ]:
def goal_loss(x_pred, goal, nq, n_col):
    dtc = DiffFrameTranslationCost.apply
    dvc = DiffFrameVelocityCost.apply

    loss  = 6e1*torch.linalg.norm(dtc(x_pred[-2*nq:], model, data, f_id) - goal)
    loss += 3e0*torch.linalg.norm(dvc(x_pred[-2*nq:], torch.zeros(nq), model, data, f_id)) # asking for zero velocity
    loss += 1e-2*torch.linalg.norm(x_pred[-2*nq:-nq]) # joint regularization
    
    for i in range(n_col):    
        loss += 2e0 * torch.linalg.norm(dtc(x_pred[(3*i)*nq: (3*i+2)*nq], model, data, f_id) - goal)
        loss += 5e-1*torch.linalg.norm(dvc(x_pred[(3*i)*nq: (3*i+2)*nq], x_pred[(3*i+2)*nq:(3*i+3)*nq], model, data, f_id)) # asking for zero velocity
        loss += 1e-2*torch.linalg.norm(x_pred[(3*i+2)*nq: (3*i+3)*nq]) # control regularization
        loss += 2e-1*torch.linalg.norm(x_pred[(3*i+1)*nq: (3*i+2)*nq]) # velocity regularization
        loss += 3e-3*torch.linalg.norm(x_pred[(3*i)*nq: (3*i+1)*nq]) # joint regularization
        loss += 2e-3*torch.linalg.norm(x_pred[(3*i)*nq+3: (3*i+1)*nq])
        loss += 4e-3*torch.linalg.norm(x_pred[(3*i)*nq+5])
        
        if i < n_col - 1:
            loss += 5e-2*torch.linalg.norm(torch.subtract(x_pred[(3*i+2)*nq: (3*i+3)*nq], \
                                                          x_pred[(3*i+5)*nq: (3*i+6)*nq]))

    return loss

In [ ]:
def obstacle_loss(x_pred, obs_pos_arr, obs_rad, nq, n_col, j_id_arr):
    dtc = DiffFrameTranslationCost.apply
    loss = 0
    for obs_pos in obs_pos_arr:
        for j_id in j_id_arr:
            e = dtc(x_pred[-2*nq:], model, data, j_id) - obs_pos
            A = torch.diag(1 / (torch.tensor(obs_rad) ** 2)).double()
            dist_loss = torch.sqrt(e @ A @ e)
            loss += 5e1*torch.exp(-(dist_loss-1.0)/0.01)

    return loss

In [ ]:
def task_loss(x_pred, goal, obs_pos_arr, obs_rad, config):
    nq, n_col, j_id_arr = config.nq, config.n_col, config.j_id_arr
    lo = obstacle_loss(x_pred, obs_pos_arr, obs_rad, nq, n_col, j_id_arr)
    lg = goal_loss(x_pred, goal, nq, n_col)
    loss = lg + lo
    return loss

In [11]:
n_tasks = 500
x_train, y_train = generate_obstacle(n_tasks, task_loss, robot, wandb.config, viz)

 79%|███████▉  | 396/500 [1:04:49<14:04,  8.12s/it]

0.15173398050139453


 80%|████████  | 400/500 [1:05:24<13:19,  8.00s/it]

0.1174235389305794


 87%|████████▋ | 437/500 [1:11:27<09:15,  8.82s/it]

...

 98%|█████████▊| 489/500 [1:20:21<01:52, 10.25s/it]

0.11827616115544878


 98%|█████████▊| 491/500 [1:20:42<01:30, 10.04s/it]

0.10083286885701577


100%|██████████| 500/500 [1:22:13<00:00,  9.87s/it]


In [12]:
data_dir = '../data/'

In [13]:
n_run = run.name.split('-')[-1]
file_path = data_dir + 'data_obstacle_' + str(n_tasks) + '_' + n_run + '.pt'
data_train = list(zip(x_train, y_train))
torch.save(data_train, file_path)

In [14]:
data_artifacts = wandb.Artifact('data_obstacle', type='dataset')
data_artifacts.add_dir(data_dir)
run.log_artifact(data_artifacts)

wandb: Adding directory to artifact (./../data)... Done. 0.1s
